# 使用 Unsloth 对 DeepSeek-R1-Distill-Qwen-1.5B 模型进行 LoRA 微调

本 Notebook 展示了如何使用 `unsloth` 库对 `deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B` 模型进行高效的 QLoRA (Low-Rank Adaptation) 微调。

整个流程包括：
1.  环境准备与库导入
2.  加载预训练模型和分词器 (Tokenizer)。
3.  在微调前，对模型进行简单的推理测试。
4.  下载和格式化训练数据集
5.  使用 `unsloth` 的 `FastLanguageModel` 来为模型添加 LoRA 适配器。
6.  配置 `SFTTrainer` 监督微调训练配置。
7.  启动训练，并观察 Loss 变化情况
8.  保存微调后的模型
9.  测试训练后的生成结果

### 1. 环境准备与库导入

首先，我们需要安装并导入所有必要的库。`transformers` 用于加载模型和分词器，`unsloth` 用于高效微调，`trl` 提供了 `SFTTrainer`，而 `datasets` 用于处理数据。

**注意**: 在运行此 Notebook 之前，请确保已安装所有依赖包：

In [1]:
pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.9/503.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 92.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.9/181.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from unsloth import FastLanguageModel
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig, DataCollatorForSeq2Seq
from datasets import Dataset

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### 2. 加载预训练模型和分词器 (Tokenizer)

In [3]:
# 定义模型和一些基本参数
max_seq_length = 8192
dtype = None # None 表示自动选择 (Float16 a T4, V100, BFloat16 a Ampere)
load_in_4bit = True # 使用 4bit 量化加载

# 这是您的模型标识符，请替换为您正在使用的模型
# 例如："qwen-1.5b_lora_model"
# model_name = "qwen-1.5b_lora_model"
# model_name = "unsloth/DeepSeek-R1-Distill-Qwen-1.5B"
model_name = "unsloth/DeepSeek-R1-Distill-Qwen-14B-unsloth-bnb-4bit"

# 这一步会返回一个经过 Unsloth 优化的模型和一个分词器
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.8.5: Fast Qwen2 patching. Transformers: 4.55.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.35G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

### 3. 微调前推理测试

在对模型进行任何修改之前，我们先用它来生成一段文本，看看原始模型的表现如何。这可以作为我们微调效果的基准参考。

In [4]:
# 模型推理的 Prompt 模板
inference_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>{}
"""

In [5]:
FastLanguageModel.for_inference(model)

question = "男，28岁，程序员，最近一周每天工作到半夜，感觉头晕、脖子疼，有时候还恶心。"

inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")
attention_mask = inputs.input_ids.ne(tokenizer.pad_token_id).long().to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

In [6]:
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [7]:
print(response[0].split("### Response:")[1])


<think>
嗯，我现在要处理一个关于28岁程序员的医疗问题。他最近一周每天工作到半夜，现在感觉头晕、脖子疼，有时候还恶心。我需要帮他分析可能的原因，并给出建议。

首先，我得考虑他的工作习惯。程序员通常久坐，长时间对着电脑，这可能导致肌肉疲劳，尤其是颈部和肩部。再加上每天工作到半夜，睡眠不足，可能会影响整体健康。

头晕和脖子疼可能与颈椎问题有关。长时间保持一个姿势，容易导致肌肉紧张，甚至颈椎病。此外，缺乏运动和血液循环不畅也可能加剧症状。

恶心可能与多种因素有关，比如压力大、焦虑，或者颈椎问题影响到神经。也有可能是由于长时间盯着屏幕导致的眼睛疲劳，或者是血压问题。

我需要考虑其他可能的原因，比如脱水、低血糖，或者是其他内科疾病，比如内耳问题。不过，考虑到他的工作性质，颈椎病和生活习惯的问题可能性更大。

接下来，我应该建议他调整生活习惯，比如改善坐姿，定时活动，保证充足睡眠，适量运动，保持良好的饮食和水分摄入。同时，建议他暂时减少电子设备的使用时间，观察症状是否缓解。

如果这些措施没有效果，或者症状加重，比如出现严重的头痛、视力模糊、呕吐，或者下肢无力，应该及时就医，进行进一步的检查，比如颈椎的影像学检查，或者血压、血糖的检测。

最后，我需要以专业但易懂的语言表达这些建议，同时保持语气友好，让他感到被关心和支持。
</think>

根据你的描述，这位28岁的程序员可能面临以下健康问题：

1. **颈椎问题**：长时间久坐，尤其是每天工作到半夜，可能导致颈部肌肉疲劳和颈椎病。这会引起头晕、脖子疼，甚至恶心。

2. **生活习惯问题**：睡眠不足、缺乏运动、饮食不规律可能导致身体状况恶化。

3. **其他原因**：如脱水、低血糖、眼睛疲劳或血压问题也可能导致这些症状。

**建议**：
- 改善坐姿，定时活动，避免长时间保持同一姿势。
- 保证充足睡眠，调整工作时间，避免熬夜。
- 适量运动，促进血液循环。
- 保持良好的饮食习惯，多喝水，避免长时间使用电子设备。
- 暂时减少电子设备使用时间，观察症状变化。

如果症状持续或加重，请及时就医，进行进一步检查。


---

### 4. 下载和格式化训练数据集


医学推理数据集：https://huggingface.co/datasets/FreedomIntelligence/medical-o1-reasoning-SFT/viewer/zh


In [8]:
# 模型训练的 Prompt 模板
train_prompt = """以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
{}

### Response:
<think>
{}
</think>
{}
"""

In [9]:
EOS_TOKEN = tokenizer.eos_token # 添加 EOS Token

def formatting_prompts_func(examples):
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    for input, cot, output in zip(inputs, cots, outputs):
        # 将 EOS Token 添加到样本最后
        text = train_prompt.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "zh", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft_Chinese.json:   0%|          | 0.00/50.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20171 [00:00<?, ? examples/s]

Map:   0%|          | 0/20171 [00:00<?, ? examples/s]

In [10]:
dataset[0]["text"]

'以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。\n请撰写一份恰当的回复，以完成该请求。\n在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。\n\n\n### Instruction:\n你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。\n请回答以下医学问题。\n\n### Question:\n根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？\n\n### Response:\n<think>\n这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。\n\n用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。\n\n但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。\n\n等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。\n\n仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。\n\n回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？\n\n再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。\n\n看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。\n\n不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。\n\n

In [11]:
from IPython.display import display, Markdown

display(Markdown(dataset[0]["text"]))

以下是一条描述任务的指令，并配有一个提供进一步上下文的输入。
请撰写一份恰当的回复，以完成该请求。
在回答之前，请仔细思考该问题，并构建一个分步的思考过程，以确保回应的逻辑严谨和内容准确。


### Instruction:
你是一位医学专家，在临床推理、诊断学和治疗规划方面拥有深厚的专业知识。
请回答以下医学问题。

### Question:
根据描述，一个1岁的孩子在夏季头皮出现多处小结节，长期不愈合，且现在疮大如梅，溃破流脓，口不收敛，头皮下有空洞，患处皮肤增厚。这种病症在中医中诊断为什么病？

### Response:
<think>
这个小孩子在夏天头皮上长了些小结节，一直都没好，后来变成了脓包，流了好多脓。想想夏天那么热，可能和湿热有关。才一岁的小孩，免疫力本来就不强，夏天的湿热没准就侵袭了身体。

用中医的角度来看，出现小结节、再加上长期不愈合，这些症状让我想到了头疮。小孩子最容易得这些皮肤病，主要因为湿热在体表郁结。

但再看看，头皮下还有空洞，这可能不止是简单的头疮。看起来病情挺严重的，也许是脓肿没治好。这样的情况中医中有时候叫做禿疮或者湿疮，也可能是另一种情况。

等一下，头皮上的空洞和皮肤增厚更像是疾病已经深入到头皮下，这是不是说明有可能是流注或瘰疬？这些名字常描述头部或颈部的严重感染，特别是有化脓不愈合，又形成通道或空洞的情况。

仔细想想，我怎么感觉这些症状更贴近瘰疬的表现？尤其考虑到孩子的年纪和夏天发生的季节性因素，湿热可能是主因，但可能也有火毒或者痰湿造成的滞留。

回到基本的症状描述上看，这种长期不愈合又复杂的状况，如果结合中医更偏重的病名，是不是有可能是涉及更深层次的感染？

再考虑一下，这应该不是单纯的瘰疬，得仔细分析头皮增厚并出现空洞这样的严重症状。中医里头，这样的表现可能更符合‘蚀疮’或‘头疽’。这些病名通常描述头部严重感染后的溃烂和组织坏死。

看看季节和孩子的体质，夏天又湿又热，外邪很容易侵入头部，对孩子这么弱的免疫系统简直就是挑战。头疽这个病名听起来真是切合，因为它描述的感染严重，溃烂到出现空洞。

不过，仔细琢磨后发现，还有个病名似乎更为合适，叫做‘蝼蛄疖’，这病在中医里专指像这种严重感染并伴有深部空洞的情况。它也涵盖了化脓和皮肤增厚这些症状。

哦，该不会是夏季湿热，导致湿毒入侵，孩子的体质不能御，其病情发展成这样的感染？综合分析后我觉得‘蝼蛄疖’这个病名真是相当符合。
</think>
从中医的角度来看，你所描述的症状符合“蝼蛄疖”的病症。这种病症通常发生在头皮，表现为多处结节，溃破流脓，形成空洞，患处皮肤增厚且长期不愈合。湿热较重的夏季更容易导致这种病症的发展，特别是在免疫力较弱的儿童身上。建议结合中医的清热解毒、祛湿消肿的治疗方法进行处理，并配合专业的医疗建议进行详细诊断和治疗。
<｜end▁of▁sentence｜>

### 5. 使用 Unsloth 添加 LoRA 适配器

这是使用 `unsloth` 的核心步骤。我们调用 `FastLanguageModel.get_peft_model`，它会非常高效地为模型注入 LoRA 模块。

- `r`: LoRA 的秩 (rank)，是控制模型复杂度和参数量的关键超参数。
- `target_modules`: 指定要在哪些线性层（如注意力机制的 q, k, v, o 投影层）上应用 LoRA。
- `lora_alpha`: LoRA 的缩放因子，通常设置为 `r` 的两倍或与 `r` 相同。
- `use_gradient_checkpointing`: 一种节省显存的技术，对于训练大模型至关重要。

In [12]:
# 因为 `model` 对象现在是由 Unsloth 创建的，它包含了所有必需的属性
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=[
      "q_proj",
      "k_proj",
      "v_proj",
      "o_proj",
      "gate_proj",
      "up_proj",
      "down_proj",
    ],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=1432,
    use_rslora=False,
    loftq_config=None,
)
# 检查模型结构，确认 LoRA 适配器已添加
print(model)

Unsloth 2025.8.5 patched 48 layers with 48 QKV layers, 48 O layers and 48 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(152064, 5120, padding_idx=151654)
        (layers): ModuleList(
          (0-4): 5 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=5120, out_features=5120, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=5120, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=5120, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

### 6. 配置 SFTTrainer

`SFTTrainer` (Supervised Fine-tuning Trainer) 是一个专门用于指令微调的训练器。我们需要配置 `TrainingArguments` 来指定所有的训练参数，如批量大小、学习率、优化器等。

In [13]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 64,
        gradient_accumulation_steps = 2,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 1432,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/20171 [00:00<?, ? examples/s]

### 7. 开始训练

一切准备就绪后，调用 `trainer.train()` 即可开始微调过程。训练结束后，会返回包含训练统计信息（如训练损失）的对象。

In [14]:
trainer_stats = trainer.train()

# 打印训练统计信息
print(trainer_stats)

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 64 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (64 x 2 x 1) = 128
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 57 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (57 x 2 x 1) = 114
 "-____-"     Trainable parameters = 68,812,800 of 14,838,846,464 (0.46% trained)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 20,171 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 51 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 

Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.670300
2,2.567000
3,2.597400
4,2.542700
5,2.521800
6,2.438900
7,2.324300
8,2.149700
9,1.996900
10,1.984600


TrainOutput(global_step=60, training_loss=1.7830180088678995, metrics={'train_runtime': 2081.4753, 'train_samples_per_second': 2.306, 'train_steps_per_second': 0.029, 'total_flos': 3.874495414611149e+17, 'train_loss': 1.7830180088678995})


### 8. 保存微调后的模型（Lora）

训练完成后，您可以再次进行推理，比较微调后的模型与原始模型的差异。如果对结果满意，可以使用 `model.save_pretrained("your_lora_adapter_path")` 来保存训练好的 LoRA 适配器。

In [16]:
model.save_pretrained("qwen-14b_lora_model")

In [17]:
tokenizer.save_pretrained("qwen-14b_lora_model")

('qwen-14b_lora_model/tokenizer_config.json',
 'qwen-14b_lora_model/special_tokens_map.json',
 'qwen-14b_lora_model/chat_template.jinja',
 'qwen-14b_lora_model/tokenizer.json')

### 9. 测试训练后的生成结果

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

question="一个患有急性阑尾炎的病人已经发病5天，腹痛稍有减轻但仍然发热，在体检时发现右下腹有压痛的包块，此时应如何处理？", # Question
inputs = tokenizer([inference_prompt.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1000,
    use_cache=True,
)

In [19]:
output = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(output[0].split("### Response:")[1])


<think>
这位病人已经患病5天了，腹痛虽然有所减轻，但还是发热。嗯，听起来好像情况没有完全好转。再看看右下腹的压痛包块，这可能意味着阑尾炎还在继续发展或者有其他并发症。

首先，阑尾炎一般在发病后的几天里会逐渐加重，但这个病人的症状却稍有减轻，这让我有点困惑。是不是炎症有缓解的可能？或者是不是有其他情况？比如阑尾穿孔或者阑尾周围脓肿？

阑尾穿孔会导致剧烈疼痛和发热，但这个病人并没有这么剧烈的疼痛，只是腹痛减轻了。所以，穿孔的可能性似乎不大。那么，考虑阑尾周围脓肿的可能性呢？这种情况下，病人可能会有包块，同时发热和疼痛。

再想想，阑尾周围脓肿会导致局部炎症反应，可能会有包块，符合这个病人的体检结果。那处理这种情况下，通常会怎么做呢？

一般来说，如果怀疑有脓肿，通常会建议病人进行影像学检查，比如超声波或者CT扫描，以确认诊断。如果确认有脓肿，可能需要进行手术引流或者在超声引导下穿刺引流。

但是，这个病人已经患病5天，而且症状没有完全缓解，考虑到阑尾炎的并发症，可能需要进一步的干预。

所以，针对这种情况，我建议首先进行影像学检查，以确认是否有脓肿。如果有，可能需要手术干预，以防止病情进一步恶化。

当然，这只是初步的思考，具体情况还需要根据检查结果来决定。
</think>
对于一位患有急性阑尾炎的病人，已经发病5天，腹痛稍有减轻但仍然发热，并且在体检时发现右下腹有压痛的包块，这种情况需要谨慎处理。

首先，考虑阑尾周围脓肿的可能性。这种情况下，病人可能会出现包块，同时发热和疼痛。建议进行影像学检查，如超声波或CT扫描，以确认是否有脓肿。

如果影像学检查确认有脓肿，通常需要进行手术引流或在超声引导下进行穿刺引流。这些措施有助于防止病情进一步恶化。

然而，考虑到病人已经患病5天，且症状未完全缓解，建议及时进行影像学检查，以明确诊断并制定下一步治疗方案。根据检查结果，采取相应的干预措施，以确保病情得到适当控制。

总之，面对这种情况，影像学检查和进一步的专业评估是必要的，以确保能够做出最合适的治疗决策。



In [20]:
def generate_response(question: str, model, tokenizer, inference_prompt: str, max_new_tokens: int = 1024) -> str:
    """
    使用指定的模型和分词器为给定的医学问题生成响应。

    Args:
        question (str): 需要模型回答的医学问题。
        model: 已加载的 Unsloth/Hugging Face 模型。
        tokenizer: 对应的分词器。
        inference_prompt (str): 用于格式化输入的 f-string 模板。
        max_new_tokens (int, optional): 生成响应的最大 token 数量。默认为 1024。

    Returns:
        str: 模型生成的响应文本，已去除 prompt 部分。
    """
    # 1. 使用模板格式化输入
    prompt = inference_prompt.format(
        question, # 填充问题
        "",       # 留空，让模型生成 CoT 和 Response
    )

    # 2. 将格式化后的 prompt 进行分词，并转移到 GPU
    inputs = tokenizer([prompt], return_tensors="pt").to(model.device)

    # 3. 使用模型生成输出
    # use_cache=True 用于加速解码过程
    outputs = model.generate(
        input_ids=inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_new_tokens=max_new_tokens,
        use_cache=True,
    )

    # 4. 将生成的 token 解码为文本
    # skip_special_tokens=True 会移除像 EOS_TOKEN 这样的特殊标记
    decoded_output = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # 5. 切分字符串，只返回 "### Response:" 之后的部分
    # 使用 .split() 分割并获取响应内容，.strip() 用于去除可能存在的前后空白字符
    response_part = decoded_output.split("### Response:")
    if len(response_part) > 1:
        return response_part[1].strip()
    else:
        # 如果模型没有生成 "### Response:" 标记，则返回整个生成内容以供调试
        return decoded_output

In [21]:
my_question = "对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，请问哪一项实验室检查对了解胸水的性质更有帮助？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
嗯，这个60岁的男性患者，右侧胸疼，还有X线显示右侧肋膈角消失。嗯，这很可能是肺结核伴胸腔积液。那么，要确定这个胸水的性质，我需要想想看，有哪些实验室检查比较有用。

首先，胸水常规检查，比如颜色、pH值、蛋白、细胞数这些。这些可以帮我们初步了解胸水的性质，比如是漏出液还是渗出液。但是，我觉得可能需要更多的信息。

然后是胸水生化检查，比如乳酸脱氢酶、葡萄糖、腺苷脱氢酶这些。这些能进一步帮助我们区分结核性胸水和癌性胸水。嗯，这可能更有用，但我觉得还不够。

再想想，胸水病理学检查，比如细胞学检查和细菌培养。这些确实能更直接地确认胸水的性质，尤其是如果怀疑结核或肿瘤。但我觉得这可能有点复杂。

最后，我想到了胸水分子生物学检查，比如PCR检测结核杆菌。这能快速确认结核的存在，而且很有效。嗯，这听起来很有帮助，尤其是在诊断结核性胸水方面。

综合来看，我觉得胸水分子生物学检查，特别是PCR检测结核杆菌，对于了解胸水的性质特别有帮助。因为如果胸水是结核性的，PCR就能快速确认。嗯，这应该是个不错的选择。
</think>
对于一名60岁男性患者，出现右侧胸疼并在X线检查中显示右侧肋膈角消失，诊断为肺结核伴右侧胸腔积液，要了解胸水的性质，最有效的实验室检查是胸水分子生物学检查，特别是PCR检测结核杆菌。这种检查可以快速确认胸水是否为结核性，从而帮助明确诊断。因此，胸水PCR检测结核杆菌对于了解胸水的性质最有帮助。


In [22]:
my_question = "对于一名 28 岁的男性患者，工作是程序员，常年熬夜，最近突然感觉头晕目眩，甚至有点恶心。请问有可能是什么疾病？"

response = generate_response(my_question, model, tokenizer, inference_prompt)
print("==================== 模型回答 ====================")
print(response)

==================== 模型回答 ====================
<think>
这位28岁的程序员，常年熬夜，最近头晕目眩，还有点恶心，这些症状让我想到可能是什么原因呢？熬夜肯定会影响身体的正常节律，长期下去可能会导致各种健康问题。嗯，头晕和恶心，这让我想到可能是由于熬夜导致的睡眠不足引起的。

可是，这些症状也可能是其他原因，比如颈椎问题。毕竟，程序员经常对着电脑，姿势可能不好，长时间保持一个姿势，颈椎的压力可能会很大，引发头晕和恶心。

或者，也有可能是贫血。贫血也会引起头晕，但通常还会伴随着乏力、面色苍白等其他症状。不过，患者没有提到这些，所以贫血的可能性可能不是主要的。

还有，高血压。高血压也会引起头晕，尤其是突然头晕，但通常还会有其他症状，比如心悸、头痛，患者也没有提到这些，所以高血压的可能性可能也不大。

不过，还有一个可能性，那就是颈椎病。长期的不良姿势和久坐，加上熬夜，颈椎的压力很大。颈椎病确实会引起头晕和恶心，尤其是当颈椎受到压迫时，可能会导致这些症状。

总结一下，患者的症状和生活习惯，似乎颈椎病是一个比较合理的解释。毕竟，程序员的工作习惯和生活方式，加上这些症状，都指向颈椎的问题。所以，我倾向于认为，这位患者的头晕目眩和恶心，很可能是由于颈椎病引起的。
</think>
根据患者的症状和生活习惯，头晕目眩和恶心很可能是由于颈椎病引起的。这是因为程序员的工作习惯通常需要长时间面对电脑，姿势可能不佳，加上长期熬夜，颈椎受到的压力很大。颈椎病可以导致头晕和恶心，尤其是在颈椎受到压迫时。因此，建议患者尽快咨询医生，进行详细的颈椎检查，以确定具体的病因，并获得相应的治疗建议。


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [24]:
!ls -lah /content

total 44K
drwxr-xr-x 1 root root 4.0K Sep 16 21:28 .
drwxr-xr-x 1 root root 4.0K Sep 16 20:31 ..
drwxr-xr-x 4 root root 4.0K Sep 15 17:50 .config
drwx------ 6 root root 4.0K Sep 16 21:28 drive
drwxr-xr-x 4 root root 4.0K Sep 16 20:48 huggingface_tokenizers_cache
drwxr-xr-x 3 root root 4.0K Sep 16 21:26 outputs
drwxr-xr-x 2 root root 4.0K Sep 16 21:26 qwen-14b_lora_model
drwxr-xr-x 2 root root 4.0K Sep 16 21:26 qwen-7.5b_lora_model
-rw-r--r-- 1 root root  161 Sep 16 20:46 requirements-colab.txt
drwxr-xr-x 1 root root 4.0K Sep 15 17:50 sample_data
drwxr-xr-x 3 root root 4.0K Sep 16 20:47 unsloth_compiled_cache


In [25]:
!cd /content && zip -r qwen-14b_lora_model.zip qwen-14b_lora_model

  adding: qwen-14b_lora_model/ (stored 0%)
  adding: qwen-14b_lora_model/adapter_config.json (deflated 57%)
  adding: qwen-14b_lora_model/special_tokens_map.json (deflated 70%)
  adding: qwen-14b_lora_model/chat_template.jinja (deflated 75%)
  adding: qwen-14b_lora_model/tokenizer_config.json (deflated 88%)
  adding: qwen-14b_lora_model/tokenizer.json (deflated 81%)
  adding: qwen-14b_lora_model/README.md (deflated 65%)
  adding: qwen-14b_lora_model/adapter_model.safetensors (deflated 7%)
